# read in data


In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/genetic-engineering-attribution-challenge/code/')

In [4]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
train_frac = .8
DATA_DIR = Path.cwd().parent / 'data/train_frac_{}'.format(int(train_frac*100))
predict = True

In [5]:
DATA_DIR

PosixPath('/content/drive/My Drive/Colab Notebooks/genetic-engineering-attribution-challenge/data/train_frac_80')

In [6]:
!pip install datatable

     |████████████████████████████████| 83.7MB 55kB/s 


In [7]:
import datatable as dt

In [8]:
y_probs_seq = dt.fread("{}/y_probs_seq.csv".format(DATA_DIR))
y_probs_seq = y_probs_seq.to_numpy()
y_probs_seq.shape

(50413, 1314)

In [9]:
y_val_probs_seq = dt.fread("{}/y_val_probs_seq.csv".format(DATA_DIR))
y_val_probs_seq = y_val_probs_seq.to_numpy()
y_val_probs_seq.shape

(12604, 1314)

In [10]:
y_probs_cnn = dt.fread("{}/y_probs_cnn.csv".format(DATA_DIR))
y_probs_cnn = y_probs_cnn.to_numpy()
y_probs_cnn.shape

(50413, 1314)

In [11]:
y_val_probs_cnn = dt.fread("{}/y_val_probs_cnn.csv".format(DATA_DIR))
y_val_probs_cnn = y_val_probs_cnn.to_numpy()
y_val_probs_cnn.shape

(12604, 1314)

In [12]:
y_probs_rnn = dt.fread("{}/y_probs_rnn.csv".format(DATA_DIR))
y_probs_rnn = y_probs_rnn.to_numpy()
y_probs_rnn.shape

(50413, 1314)

In [13]:
y_val_probs_rnn = dt.fread("{}/y_val_probs_rnn.csv".format(DATA_DIR))
y_val_probs_rnn = y_val_probs_rnn.to_numpy()
y_val_probs_rnn.shape

(12604, 1314)

In [14]:
y = dt.fread(DATA_DIR / 'y.csv')#, max_nrows=30000)
y = y.to_numpy()
print(y.shape)
y_val = dt.fread(DATA_DIR / 'y_val.csv')#, max_nrows=3000)
y_val = y_val.to_numpy()
print(y_val.shape)

(50413, 1314)
(12604, 1314)


In [15]:
if predict:
  y_probs_seq = np.concatenate([y_probs_seq, y_val_probs_seq], axis=0)
  y_probs_cnn = np.concatenate([y_probs_cnn, y_val_probs_cnn], axis=0)
  y_probs_rnn = np.concatenate([y_probs_rnn, y_val_probs_rnn], axis=0)
  y = np.concatenate([y, y_val], axis=0)

In [16]:
y_probs_list = [y_probs_seq, y_probs_cnn, y_probs_rnn]
y_val_probs_list = [y_val_probs_seq, y_val_probs_cnn, y_val_probs_rnn]

# correlation of models

In [19]:
def group_y(y_probs, y):


  correct_probs = np.multiply(y_probs, y)

  y_probs_df = pd.DataFrame({'probs': correct_probs.sum(axis=1), 
                            'class':np.argmax(y, axis=1)})
  y_grouped_df = y_probs_df.groupby('class'). \
    agg(mean = ('probs','mean'),
        count = ('probs','count'))
  y_grouped_df['amount'] = (1 - y_grouped_df['mean']) * y_grouped_df['count']
  
  return y_grouped_df

In [24]:
y_probs_grouped_list = [group_y(y_probs, y) for y_probs in y_probs_list]
y_probs_grouped_all = pd.concat(y_probs_grouped_list, axis=1)
y_probs_grouped_all.columns = ['mean_1',	'count_1',	'amount_1', 'mean_2',	'count_2',	'amount_2', ]
y_probs_grouped_all.corr()

,mean,count,amount,mean,count,amount,mean,count,amount
mean,1.000000,0.090436,-0.012633,0.639784,0.090436,-0.012860,0.061302,0.090436,0.098047
count,0.090436,1.000000,0.917535,0.075020,1.000000,0.912313,0.640962,1.000000,0.999292
amount,-0.012633,0.917535,1.000000,0.002531,0.917535,0.989681,0.568723,0.917535,0.903689
mean,0.639784,0.075020,0.002531,1.000000,0.075020,-0.053997,0.051631,0.075020,0.080296
count,0.090436,1.000000,0.917535,0.075020,1.000000,0.912313,0.640962,1.000000,0.999292
amount,-0.012860,0.912313,0.989681,-0.053997,0.912313,1.000000,0.564415,0.912313,0.900027
mean,0.061302,0.640962,0.568723,0.051631,0.640962,0.564415,1.000000,0.640962,0.641789
count,0.090436,1.000000,0.917535,0.075020,1.000000,0.912313,0.640962,1.000000,0.999292
amount,0.098047,0.999292,0.903689,0.080296,0.999292,0.900027,0.641789,0.999292,1.000000


In [62]:
model = combined_model()
model.fit(y_probs_list, y, beta=1)
model.evaluate(y_val_probs_list, y_val) #0.9249444620755316

0.9249444620755316

# custom model

In [61]:
class combined_model:

  def fit(self, y_probs_list, y, alpha_list=None, beta=1):#y_seq, y_cnn, y_rnn, y, alpha=.5, beta=1):

    total_sums = None
    sums_dict = {}

    for i in range(len(y_probs_list)):
      y_probs = y_probs_list[i]
      correct_probs = np.multiply(y_probs, y)
      sums_dict[i] = (correct_probs.sum(axis=0)) ** beta

      if total_sums is None:
        total_sums = sums_dict[i]
      else:
        total_sums = total_sums + sums_dict[i]

    if alpha_list is None:
      alpha_list = [1.0/len(y_probs_list)] * len(y_probs_list)

    self.weights = {}
    default_alpha = 1.0/len(y_probs_list)

    for i in range(len(y_probs_list)):
      self.weights[i] = alpha_list[i] / default_alpha * sums_dict[i] / total_sums

  def evaluate(self, y_probs_list, y):#y_seq, y_cnn, y):

    y_probs = None
    for i in range(len(y_probs_list)):
      if y_probs is None:
        y_probs = np.multiply(y_probs_list[i], self.weights[i])
      else:
        y_probs = y_probs + np.multiply(y_probs_list[i], self.weights[i])

    y_probs = y_probs/y_probs.sum(axis=1)[:,None]

    top10_idx = np.argpartition(y_probs, -10, axis=1)[:, -10:]
    y_idx = np.argpartition(y, -1, axis=1)[:, -1:]
    top_10_accuracy = (top10_idx == y_idx).any(axis=1).mean()

    return top_10_accuracy

  def predict(self, y_probs_list):#y_seq, y_cnn):

    y_probs = None
    for i in range(len(y_probs_list)):
      if y_probs is None:
        y_probs = np.multiply(y_probs_list[i], self.weights[i])
      else:
        y_probs = y_probs + np.multiply(y_probs_list[i], self.weights[i])
    # y_probs = np.multiply(y_seq, self.weights_seq) + np.multiply(y_cnn, self.weights_cnn)
    y_probs = y_probs/y_probs.sum(axis=1)[:,None]

    return y_probs

  def fit_evaluate(self, y_probs_list, y, y_val_probs_list, y_val, alpha_list, beta):#y_seq, y_cnn, y, y_val_seq, y_val_cnn, y_val, alpha, beta):

    self.fit(y_probs_list, y, alpha_list, beta)#y_seq, y_cnn, y, alpha, beta)

    return self.evaluate(y_val_probs_list, y_val)#y_val_seq, y_val_cnn, y_val)

  def tune(self,
           y_probs_list, y, y_val_probs_list, y_val, 
           alpha_1_range=np.linspace(0.45,.75,30), 
           alpha_2_range=np.linspace(0.15,.24,9), 
           beta_range=np.linspace(1,15,15)):#alpha_range=np.linspace(0.2,.8,60), beta_range=np.linspace(1,50,25), y_seq=y_probs_seq, y_cnn=y_probs_cnn, y=y, y_val_seq=y_val_probs_seq, y_val_cnn=y_val_probs_cnn, y_val=y_val):


    results_dict = {}
    results_dict['alpha_1'] = []
    results_dict['alpha_2'] = []
    results_dict['alpha_3'] = []
    results_dict['beta'] = []
    results_dict['result'] = []
    for alpha_1 in alpha_1_range:
      for alpha_2 in alpha_2_range:
        for beta in beta_range:
          results_dict['alpha_1'].append(alpha_1)
          print(alpha_1)
          results_dict['alpha_2'].append(alpha_2)
          print(alpha_2)
          alpha_3 = 1 - alpha_1 - alpha_2
          results_dict['alpha_3'].append(alpha_3)
          print(alpha_3)
          results_dict['beta'].append(beta)
          print(beta)

          result = model.fit_evaluate(y_probs_list, y, y_val_probs_list, y_val, 
                                      [alpha_1, alpha_2, alpha_3], beta)

          results_dict['result'].append(result)
          print('result {}'.format(result))

    return results_dict



In [23]:
model = combined_model()
results_dict = model.tune(y_probs_list, y, y_val_probs_list, y_val)

Streaming output truncated to the last 5000 lines.
0.20625
0.11616379310344832
6.0
result 0.9239130434782609
0.6775862068965517
0.20625
0.11616379310344832
7.0
result 0.9235163440177722
0.6775862068965517
0.20625
0.11616379310344832
8.0
result 0.9231196445572833
0.6775862068965517
0.20625
0.11616379310344832
9.0
result 0.9224849254205014
0.6775862068965517
0.20625
0.11616379310344832
10.0
result 0.9217708663916218
0.6775862068965517
0.20625
0.11616379310344832
11.0
result 0.9214535068232307
0.6775862068965517
0.20625
0.11616379310344832
12.0
result 0.9212948270390352
0.6775862068965517
0.20625
0.11616379310344832
13.0
result 0.9211361472548397
0.6775862068965517
0.20625
0.11616379310344832
14.0
result 0.9204220882259601
0.6775862068965517
0.20625
0.11616379310344832
15.0
result 0.9197080291970803
0.6775862068965517
0.2175
0.10491379310344831
1.0
result 0.9274833386226595
0.6775862068965517
0.2175
0.10491379310344831
2.0
result 0.9262932402411933
0.6775862068965517
0.2175
0.104913793103

In [19]:
0.6534375
0.18937500000000002
0.15718749999999998
1.0
result 0.9300222151697873

alpha_1	alpha_2	alpha_3	beta	result
3915	0.750000	0.15000	0.100000	1.0	0.927880

0.45
0.22241379310344828
0.32758620689655177
3.0416666666666665
result 0.9281973976515392

SyntaxError: ignored

In [ ]:
results_df = pd.DataFrame(results_dict)
results_df.sort_values('result', ascending=False)

In [ ]:
def combined_model_fit_eval(x, y_seq=y_probs_seq, y_cnn=y_probs_cnn, y=y, y_val_seq=y_val_probs_seq, y_val_cnn=y_val_probs_cnn, y_val=y_val):

  model = combined_model()
  alpha = x[0]
  beta = x[1]
  return -100 * model.fit_evaluate(y_seq, y_cnn, y, y_val_seq, y_val_cnn, y_val,alpha, beta)

from scipy.optimize import minimize
minimize(combined_model_fit_eval, [.5,10], bounds=[(0,1),(1,100)], method='trust-constr')

In [56]:
model = combined_model()
model.fit(y_probs_list, y, 
          alpha_list=[.75,.15,.1], beta=1.0)#y_probs_seq, y_probs_cnn, y, 0.7,9)
model.evaluate(y_val_probs_list, y_val)

0.9365280863218026

In [ ]:
with open('{}/models/latest_combined_model.pkl'.format(DATA_DIR), 'wb') as output:
  pickle.dump(model, output, pickle.HIGHEST_PROTOCOL)

from datetime import datetime
timestamp = datetime.now().strftime("%d-%m-%Y %H_%M_%S")
with open('{}/models/{}_combined_model.pkl'.format(DATA_DIR, timestamp), 'wb') as output:
  pickle.dump(model, output, pickle.HIGHEST_PROTOCOL)

# test data

In [57]:
y_test_probs_seq = dt.fread("{}/y_test_probs_seq.csv".format(DATA_DIR))
y_test_probs_seq = y_test_probs_seq.to_numpy()
y_test_probs_seq.shape

(18816, 1314)

In [58]:
y_test_probs_cnn = dt.fread("{}/y_test_probs_cnn.csv".format(DATA_DIR))
y_test_probs_cnn = y_test_probs_cnn.to_numpy()
y_test_probs_cnn.shape

(18816, 1314)

In [59]:
y_test_probs_rnn = dt.fread("{}/y_test_probs_rnn.csv".format(DATA_DIR))
y_test_probs_rnn = y_test_probs_rnn.to_numpy()
y_test_probs_rnn.shape

(18816, 1314)

In [60]:
y_test_probs = model.predict([y_test_probs_seq, y_test_probs_cnn, y_test_probs_rnn])
y_test_probs.shape

(18816, 1314)

# submission

In [61]:
submission_format = pd.read_csv(DATA_DIR.parent / 'submissions/submission_format.csv', index_col='sequence_id')

In [62]:
assert submission_format.shape == y_test_probs.shape

In [63]:
my_submission = pd.DataFrame(data=y_test_probs, 
                             columns=submission_format.columns, 
                             index=submission_format.index)
my_submission.head(5)

00Q4V31T      012VT4JK      028IO5W2      03GRNN7N  \
sequence_id                                                           
E0VFT        8.897258e-09  1.174439e-08  3.613947e-08  2.042221e-08   
TTRK5        4.848937e-08  1.373128e-09  3.326268e-08  4.138037e-08   
2Z7FZ        3.347081e-06  9.213884e-07  2.572546e-06  8.305573e-06   
VJI6E        2.148815e-06  1.905234e-07  7.371762e-07  5.881984e-06   
721FI        5.439110e-06  5.153279e-07  5.902481e-07  3.507488e-06   

                 03Y3W51H      09MQV1TY      0A4AHRCT      0A9M05NC  \
sequence_id                                                           
E0VFT        1.344055e-06  1.236333e-08  9.178810e-08  1.348403e-08   
TTRK5        2.543192e-07  3.760395e-08  2.688414e-09  1.030024e-07   
2Z7FZ        4.418329e-07  6.494788e-05  2.030218e-04  2.874916e-05   
VJI6E        2.222524e-05  4.541838e-08  1.036028e-07  1.261316e-07   
721FI        7.390329e-06  5.046349e-07  3.583649e-06  3.919840e-06   

                 0B9GCUVV      0CL7QVG8      0CML4B5I      0DTHTJLJ  \
sequence_id                                                           
E0VFT        3.907706e-08  9.525316e-08  1.812862e-08  1.120886e-08   
TTRK5        4.313268e-08  1.203369e-08  1.818791e-09  1.610544e-08   
2Z7FZ        3.751631e-06  1.805943e-06  2.462904e-05  3.300831e-06   
VJI6E        1.620982e-03  3.899342e-06  1.441052e-05  6.448393e-06   
721FI        2.140638e-05  6.759322e-07  1.908694e-05  1.711110e-06   

                 0FFBBVE1      0HWCWFNU      0L3Y6ZB2      0M44GDO8  \
sequence_id                                                           
E0VFT        2.516853e-07  6.794218e-08  7.804801e-10  2.010275e-09   
TTRK5        3.429099e-10  1.451972e-08  2.596122e-10  6.319867e-09   
2Z7FZ        2.499732e-07  3.094603e-06  3.307196e-08  1.874197e-06   
VJI6E        5.263322e-01  3.852762e-06  5.193325e-09  3.201740e-07   
721FI        1.976332e-05  3.406026e-06  3.719664e-09  2.007717e-07   

                 0MDYJM3H      0N3V9P9M      0NP55E93      0PJ91ZT6  \
sequence_id                                                           
E0VFT        1.169914e-08  1.771724e-07  8.455201e-08  7.270581e-07   
TTRK5        3.452613e-10  1.494740e-07  5.241468e-08  1.181433e-06   
2Z7FZ        7.796729e-06  2.226114e-04  5.500107e-05  5.529727e-04   
VJI6E        3.290189e-07  4.599428e-06  1.708436e-06  1.648346e-07   
721FI        1.192370e-06  1.190747e-04  2.254252e-05  1.832423e-05   

                 0R296F9R      0T2AZBD6      0URA80CN      0VRP2DI6  \
sequence_id                                                           
E0VFT        9.238928e-09  1.402380e-07  1.218504e-07  1.160854e-07   
TTRK5        2.038856e-08  2.760663e-08  5.394734e-08  4.333781e-08   
2Z7FZ        1.109768e-06  5.456236e-05  4.532706e-06  1.724837e-05   
VJI6E        8.374314e-06  2.381961e-06  2.056112e-06  7.693040e-08   
721FI        3.810992e-06  1.836001e-05  6.314930e-05  2.500058e-06   

                 0W6O08VX      0WHP4PPK      0XPTGGLP      0XS4FHP3  \
sequence_id                                                           
E0VFT        6.477340e-08  2.193278e-08  4.328267e-08  1.461827e-08   
TTRK5        1.336967e-08  1.637712e-08  9.916759e-09  1.194295e-09   
2Z7FZ        7.033899e-06  3.006660e-06  1.205990e-06  2.291743e-06   
VJI6E        7.466378e-07  1.814275e-06  2.253411e-05  2.283622e-06   
721FI        7.747800e-06  4.096868e-06  6.943329e-07  7.987934e-06   

                 0Y24J5G2      10TEBWK2      11TTDKTM      131RRHBV  \
sequence_id                                                           
E0VFT        1.175291e-08  8.742952e-09  1.085047e-07  1.527923e-07   
TTRK5        4.420604e-09  6.584043e-09  2.763441e-07  4.049988e-09   
2Z7FZ        1.820240e-06  1.281497e-04  1.313044e-04  1.124477e-06   
VJI6E        2.300482e-06  2.638910e-06  6.163660e-07  4.451057e-05   
721FI        2.688257e-06  1.469591e-06  3.787354e-06  2.805480e-05   

                 13LZE1F7      14

In [64]:
from datetime import datetime
timestamp = datetime.now().strftime("%d-%m-%Y %H_%M_%S")
my_submission.to_csv(DATA_DIR.parent / 'submissions/{}_submission.csv'.format(timestamp))
my_submission.to_csv(DATA_DIR.parent / 'submissions/latest_submission.csv')

# neural net model

## setup

In [ ]:
X = np.concatenate((y_probs_seq, y_probs_cnn), axis=1)
X.shape

(50413, 2628)

In [ ]:
del y_probs_seq, y_probs_cnn

In [ ]:
X_val = np.concatenate((y_val_probs_seq, y_val_probs_cnn), axis=1)
X_val.shape

(12604, 2628)

In [ ]:
del y_val_probs_seq, y_val_probs_cnn

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Concatenate, Input, Layer, Softmax, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from tensorflow.keras.backend import clear_session
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from sklearn.metrics import accuracy_score, auc, roc_auc_score, confusion_matrix
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import initializers

In [ ]:
top_10_accuracy = tf.keras.metrics.TopKCategoricalAccuracy(k=10)

In [ ]:
num_classes = 1314

## single model

In [ ]:
biases = np.zeros(num_classes)
weights = np.row_stack((np.identity(num_classes) * .5, np.identity(num_classes) * .5))
weights.shape

(2628, 1314)

In [ ]:
Trial summary
|-Trial ID: bfe06831547581b1dcc1333ecbbd7b92
|-Score: 0.8751983642578125
|-Best step: 0
Hyperparameters:
|-activation_function: tanh
|-batch_normalization: True
|-batch_size: 128
|-bias_GlorotNormal: True
|-dropout: False
|-dropout=True/rate: 0.016696100931225612
|-hidden_layer_size: 379
|-kernel_GlorotNormal: True
|-penalty: 3.4371369161912445e-05
|-tuner/bracket: 4
|-tuner/epochs: 30
|-tuner/initial_epoch: 15
|-tuner/round: 4
|-tuner/trial_id: d90e817dbb924717417260551ab9cdcd

In [ ]:
clear_session()
tf.random.set_seed(0)
model = Sequential()
layer = Dense(int(num_classes*1.5), 
              input_shape=(num_classes*2,), activation='sigmoid')
model.add(layer)
# model.layers[0].set_weights([weights, biases])
model.add(BatchNormalization())
model.add(Dropout(.5))
model.add(Dense(num_classes, 'softmax'))
model.compile(Adam(),
              loss='categorical_crossentropy',
              metrics=[top_10_accuracy])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1971)              5181759   
_________________________________________________________________
batch_normalization (BatchNo (None, 1971)              7884      
_________________________________________________________________
dropout (Dropout)            (None, 1971)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1314)              2591208   
Total params: 7,780,851
Trainable params: 7,776,909
Non-trainable params: 3,942
_________________________________________________________________


In [ ]:
callback = EarlyStopping(monitor='val_top_k_categorical_accuracy', patience=3)
model.fit(X, y,
      batch_size = 64,
      epochs=100, 
      verbose=1, 
      validation_data=(X_val, y_val), 
      callbacks=[callback])

Epoch 1/100
459/788 [================>.............] - ETA: 1s - loss: 0.0387 - top_k_categorical_accuracy: 0.9993

KeyboardInterrupt: ignored

## build model

In [ ]:
def build_model(hp):
  
    #  'optimizer': [Adam, Nadam, RMSprop],
    
    clear_session()
    tf.random.set_seed(0)
    model = Sequential()
    
    hidden_layer_size_min = round(np.mean([X.shape[1], y.shape[1]])*.02)
    hidden_layer_size_max = round(np.mean([X.shape[1], y.shape[1]])*.2)
    hidden_layer_size = hp.Int('hidden_layer_size',hidden_layer_size_min,hidden_layer_size_max,sampling='log')
    
    penalty = hp.Float('penalty',.00001,.01,sampling='log')
    
    activation_function = hp.Choice('activation_function', ['relu','tanh','sigmoid'])
        
    if hp.Boolean('kernel_GlorotNormal'):
      kernel_initializer = initializers.GlorotNormal(seed=0)
    else:
      kernel_initializer = initializers.GlorotUniform(seed=0)

    if hp.Boolean('bias_GlorotNormal'):
      bias_initializer = initializers.GlorotNormal(seed=1)
    else:
      bias_initializer = initializers.GlorotUniform(seed=1)

    model.add(Dense(hidden_layer_size, input_shape=(X.shape[1],), 
                    activation=activation_function,
                   kernel_regularizer=regularizers.l2(penalty),
                   bias_regularizer=regularizers.l2(penalty),
                   activity_regularizer=regularizers.l2(penalty),
                    kernel_initializer=kernel_initializer,
                    bias_initializer=bias_initializer))

    if hp.Boolean('batch_normalization'):
      model.add(BatchNormalization())

    if hp.Boolean('dropout'):
      with hp.conditional_scope('dropout', True):
        rate = hp.Float('rate',.001,.5,sampling='log')
        model.add(Dropout(rate))
    
    model.add(Dense(y.shape[1], activation='softmax'))

    # learning_rate = hp.Float('learning_rate', .00001,.1,sampling='log')
    # beta_1 = 1 - hp.Float('one_minus_beta_1', .001, .2,sampling='log')
    # beta_2 = 1 - hp.Float('one_minus_beta_2', .00001, .02,sampling='log')
    
    model.compile(Adam(),#learning_rate, beta_1, beta_2),
                  loss='categorical_crossentropy',
                  metrics=[top_10_accuracy])
    
    return model

## tuning

In [ ]:
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 4.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=194acea4272cd25114289b241b3c770b92f3c6de22264d7a201ec89bccf2a2e3
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=2733101b671667b60399108d1705f59c619a98819221dc517522c3a929a81de3
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import kerastuner as kt
from kerastuner import Objective

class MyTuner(kt.tuners.Hyperband):#(kt.engine.tuner.Tuner):
  
  def run_trial(self, trial, *args, **kwargs):
    # You can add additional HyperParameters for preprocessing and custom training loops
    # via overriding `run_trial`
    kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 16, 2048, step=16)

    super(MyTuner, self).run_trial(trial, *args, **kwargs)

 # Uses same argments as the BayesianOptimization Tuner.
tuner = MyTuner(
    hypermodel=build_model,
    # oracle=kt.oracles.BayesianOptimization(
    #   objective=Objective('val_top_k_categorical_accuracy', 'max'),
    #   max_trials=1000,
    #   seed=0,
    #   beta=10),
    objective=Objective('val_top_k_categorical_accuracy', 'max'),
    max_epochs=30,
    factor=2,
    hyperband_iterations=3,
    seed = 0,
    directory='tuning',
    project_name='sequential',
    overwrite=True)

callback = EarlyStopping(monitor='val_top_k_categorical_accuracy', patience=5)

In [ ]:
tuner.search(X, y,
             validation_data=(X_val, y_val),
             verbose=0,
             callbacks=[callback])

INFO:tensorflow:Oracle triggered exit


In [ ]:
Trial summary
|-Trial ID: bfe06831547581b1dcc1333ecbbd7b92
|-Score: 0.8751983642578125
|-Best step: 0
Hyperparameters:
|-activation_function: tanh
|-batch_normalization: True
|-batch_size: 128
|-bias_GlorotNormal: True
|-dropout: False
|-dropout=True/rate: 0.016696100931225612
|-hidden_layer_size: 379
|-kernel_GlorotNormal: True
|-penalty: 3.4371369161912445e-05
|-tuner/bracket: 4
|-tuner/epochs: 30
|-tuner/initial_epoch: 15
|-tuner/round: 4
|-tuner/trial_id: d90e817dbb924717417260551ab9cdcd